In [1]:
import numpy as np

**Module** - это абстрактный класс определяющий необходимые для нейронной сети методы. В этой части не нужно ничего писать, просто читайте комментарии.

In [3]:
class Module(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True
    """
    В целом, Module это черный ящик, который принимает что-то на вход и
    выдает какой-то результат. Это результат применения метода "forward":
        
        output = module.forward(input)
    
    Также этот черный ящик должен уметь по значеню dL/dOutput подсчитывать
    производную функции ошибки по параметрам модели (dL/dW) и по входным данным
    (dL/dInput)
    
        gradInput = module.backward(input, gradOutput)
    """
    
    def forward(self, input):
        """
        Вычислает по данному input соответствующий output, сохраняя его в одноименном
        поле себя.
        """
        
        # Пример для тождественной функции: 
        
        # self.output = input
        # return self.output
        
        assert False, 'Implementation error'

    def backward(self, input, gradOutput):
        """
        Делает шаг обратного распространения ошибки.
        
        Включает в себя:
        1) подсчет градиента функции ошибки по входу (для выполнения
           шага обратного распространения ошибки предыдущего слоя)
        2) подсчет градиента функции ошибки по параметрам (для 
           выполнения шага градиентного спуска)
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput

    def updateGradInput(self, input, gradOutput):
        """
        Подсчитывает градиент функции ошибки по входу, сохраняет результат
        в поле gradInput. Размерность полученного градиента всегда строго совпадает
        с размерностью входа.
        """
        
        # Для тождественной функции:
        
        # self.gradInput = gradOutput 
        # return self.gradInput
        
        assert False, 'Implementation error'
    
    def accGradParameters(self, input, gradOutput):
        """
        Подсчитывает градиент функции потерь по параметрам
        
        Не нужно реализовывать если у модуля нет параметров (ReLU)
        """
        pass
    
    def zeroGradParameters(self): 
        """
        Обнуляет градиент
        """
        pass
        
    def getParameters(self):
        """
        Возвращает список подгоняемых (trainable) параметров.
        
        Если таковых нет, то возвращает пустой список.
        """
        return []
        
    def getGradParameters(self):
        """
        Возвращает список градиентов функции потерь по подгоняемым параметрам.
        
        Если таковых нет, то возвращает пустой список.
        """
        return []
    
    def train(self):
        """
        Переводит модуль в режим обучение. Влияет только на дропаут и batchNorm.
        """
        self.training = True
    
    def eval(self):
        """
        Отключает режим обучения. Влияет только на дропаут и batchNorm.
        """
        self.training = False
    
    def __repr__(self):
        """
        Выводит имя модуля.
        """
        return "Module"

# Sequential container

**Define** a forward and backward pass procedures.

In [4]:
class Sequential(Module):
    """
         Это контейнер, который для данного входа последовательно применяет
         все переданные в него модули (выход предущего модуля - вход следующего)
         
         Итоговое значение будет являться выходом данного контейнера.
    """
    
    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []
   
    def add(self, module):
        """
        Добавляет модуль в контейнер.
        """
        self.modules.append(module)

    def forward(self, input):
        """
        Вычислает по данному input соответствующий output, сохраняя его в одноименном
        поле себя.
        
            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})
        """

        self.inputs = []
        last_output = input
        
        for i, mod in enumerate(self.modules):
            self.inputs.append(last_output)
            last_output = mod.forward(self.inputs[-1])
            
        self.output = last_output
                
        return self.output

    def backward(self, input, gradOutput):
        """
        Реализует backward pass:
            
            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)   
            gradInput = module[0].backward(input, g_1)   
        
        """
        yinp = self.m_inputs[::-1]
        t = gradOutput
        
        for i, mod in enumerate(self.modules[::-1]):
            t = mod.backward(yinp[i], t)
                
        self.gradInput = t
        return self.gradInput
      

    def zeroGradParameters(self): 
        for module in self.modules:
            module.zeroGradParameters()
    
    def getParameters(self):
        """
        Возвращает список списков параметров каждого модуля.
        """
        return [x.getParameters() for x in self.modules]
    
    def getGradParameters(self):
        """
        Возвращает список списков градиентов функции потерь по параметрам каждого модуля.
        """
        return [x.getGradParameters() for x in self.modules]
    
    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string
    
    def __getitem__(self,x):
        return self.modules.__getitem__(x)

# Layers

- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

In [26]:
class Linear(Module):
    """
    Модуль выполняющий линейное преобразование над входом. Также известен
    как полносвязный слой.
    
    Модуль должен принимать на вход матрицу размерности (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()
       
        # This is a nice initialization
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_out, n_in))
        
        self.gradW = np.zeros_like(self.W)
        
    def dumb_forward(self, input):
        '''
        Считает выход линейного слоя по данному входу, сохраняет результат в self.output
        и вовзращает self.output.
        output[object_idx, out_neuron_idx] = 
        \sum_{in_neuron_idx} input[object_idx,n] * w_[out_neuron_idx, in_neuron_idx]
        '''
        n_obj = len(input)
        n_out, n_in = self.W.shape
        self.output = np.zeros((n_obj, n_out))
        for i_obj in range(n_obj):
            for i_out in range(n_out):
                for i_in in range(n_in):
                    self.output[i_obj, i_out] += self.W[i_out][i_in] * input[i_obj][i_in]
                
        return self.output
    
    def forward(self, input1):
        self.output = (self.W @ input1.T).T
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        assert False, "Implementation error"

    def accGradParameters(self, input, gradOutput):
        assert False, "Implementation error"
    
    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)
        
    def getParameters(self):
        return [self.W]
    
    def getGradParameters(self):
        return [self.gradW]
    
    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

This one is probably the hardest but as others only takes 5 lines of code in total. 
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [6]:
%matplotlib inline
from time import time, sleep
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

In [7]:
def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [28]:

input_size = 4
num_classes = 3
num_inputs = 5

def init_linear():
    np.random.seed(0)
    return Linear(input_size, num_classes)

def init_toy_data():
    np.random.seed(1)
    X = 10 * np.random.randn(num_inputs, input_size)
    y = np.array([0, 1, 2, 2, 1])
    return X, y

linear = init_linear()
X, y = init_toy_data()

In [29]:
output = linear.forward(X)
print('Your output:')
print(output)
print()
print('correct output:')
correct_output = np.asarray(
[[ -1.54788446,  -6.00658097,   6.39369587],
 [ -3.07885716,  -8.08969546,  11.56531411],
 [  0.19697038,  -9.59100515,   5.43998673],
 [ -0.31239372,  -5.33085678,  1.94239605],
 [ -1.66825993,   1.10786278,   0.51528511]])
print(correct_output)
print()

# The difference should be very small. We get < 1e-7
print('Difference between your scores and correct scores:')
print(rel_error(output, correct_output))

Your output:
[[ -1.54788446  -6.00658097   6.39369587]
 [ -3.07885716  -8.08969546  11.56531411]
 [  0.19697038  -9.59100515   5.43998673]
 [ -0.31239372  -5.33085678   1.94239605]
 [ -1.66825993   1.10786278   0.51528511]]

correct output:
[[ -1.54788446  -6.00658097   6.39369587]
 [ -3.07885716  -8.08969546  11.56531411]
 [  0.19697038  -9.59100515   5.43998673]
 [ -0.31239372  -5.33085678   1.94239605]
 [ -1.66825993   1.10786278   0.51528511]]

Difference between your scores and correct scores:
1.02925382778e-08


# Функции активации

ReLU **Rectified Linear Unit** реализован за вас: 

In [32]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()
    
    def updateOutput(self, input):
        self.output = np.maximum(input, 0)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput
    
    def __repr__(self):
        return "ReLU"

In [31]:
class Sigmoid(Module):
    def __init__(self):
        super(Sigmoid, self).__init__()
    
    def forward(self, input):
        self.output = 1/(1 + np.exp(-input))
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = self.output * (1 - self.output)
        return self.gradInput
    
    def __repr__(self):
        return "Sigmoid"

In [33]:
sequential = Sequential()
linear_small = Linear(n_in=2, n_out=2)
linear_small.W = np.array([[0.9, 0.3], [0.2, 0.8]])
linear_small.b = np.zeros(2)
sequential.add(linear_small)
sequential.add(Sigmoid())

In [34]:
X_small = np.array([[1, 0.5]])
output_small = sequential.forward(X_small)
print('Your output:')
print(output_small)
print()
print('correct output:')
correct_output_small = np.asarray([[0.7408, 0.6457]])
print(correct_output_small)
print()

print('Difference between your scores and correct scores:')
print(rel_error(output_small, correct_output_small))

Your output:
[[ 0.7407749   0.64565631]]

correct output:
[[ 0.7408  0.6457]]

Difference between your scores and correct scores:
3.38355680721e-05


# Criterions

Функции потерь. 

In [2]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None
        
    def forward(self, input, target):
        """
            Считает ошибку, сохраняет ее и возвращает.
        """
        assert False, "Implementation error"

    def backward(self, input, target):
        """
            Считает градиент ошибки по input, сохраняет его в gradInput
            и возвращает.
        """
        self.updateGradInput(input, target)
        return self.gradInput

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        assert False, "Implementation error"

    def __repr__(self):
        """
        Выводит название функции потерь в человекочитаемом виде.
        """
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.

In [13]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()
        
    def updateOutput(self, input, target):   
        self.output = np.sum(np.power(input - target,2)) / input.shape[0]
        return self.output 
 
    def updateGradInput(self, input, target):
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"